# HW 5 - Tim Demetriades
## Music Recommendation 1
March 27th, 2021

In [1]:
import numpy    # for arrays
import time     # for timing cells

### Part 1 - Create Output file
Here, we create `output1.txt`, which will hold the user_id, track_id, and album rating and artist rating for a given user and track. It should be 120,000 entries long, and there should be 6 entries for each user. 

First, the input and output files are setup and opened. 

- `testTrack_hierarchy.txt` is the testing file and is in the format `UserID|trackID|Album|Artist|Genre1|Genre2|Genre3 | ....`. It is 120,000 entries long.
- `trainIdx2_matrix.txt` is the training file and is in the format `userID|itemID|score`. It is over 12,400,000 entries long.
- `output1.txt` is the output file and is in the format `userID|trackID|album_score|artist_score`. It should be 120,000 entries long (same as the testing file). 

In [2]:
data_dir = './Project/data_in_matrixForm/'
file_name_test = data_dir + 'testTrack_hierarchy.txt'
file_name_train = data_dir + 'trainIdx2_matrix.txt'
output_file = data_dir + 'output1.txt'

In [3]:
# Open files
f_test = open(file_name_test)
f_train = open(file_name_train)
train_line = f_train.readline()
f_out = open(output_file, 'w')

In [69]:
# Initialize arrays to hold track_id, album_id, and artist_id for all 6 entries of a user.
track_id_vec = [0] * 6
album_id_vec = [0] * 6
artist_id_vec = [0] * 6
last_user_id = -1

In [5]:
# Initialize numpy array to hold album and artist ratings for each user
user_rating_in_train = numpy.zeros(shape=(6,3))

Now this is the main loop for Part 1. We go through each line of the testing file. First, we take each element from a line and create a list. We then grab the user_id, track_id, album_id, and artist_id from that list. 

Next, we check if a new user has been reached (we enter this if statement from the start since `last_user_id` starts as -1). 

Then we fill out vectors to hold the track, album, and artists ids for the current user. 

Once the next user is reached, we then go through each line of the training file using a while loop and do a similar process of creating a list from the line and grabbing the elements (this time we grab the user_id, item_id, and rating for each line). 

If the user_id from the training file is less than the user_id from the testing file we just return to the beginning of the while loop.

If the user_id from the training file is the same as the user_id from the testing file, we check if there is an album score and/or artist score in the training file that corresponds with an album and/or artist in the testing file. If there is, we add the score to the corresponding index in `user_rating_in_train`. 

If the user_id from the training file is greater than the user_id from the testing file, we know a new user has been reached and we can now write to the output file. We write 6 entries for each user, taking the user_id, track_id, album_score (if found, otherwise it will be 0), and artist_score (if found, otherwise it will be 0). After writing all 6 entries for the current user we break out of the while loop.

In [6]:
start_time = time.time()

for line in f_test:
    arr_test = line.strip().split('|')
    user_id = arr_test[0]    # set user
    track_id = arr_test[1]   # set track
    album_id = arr_test[2]   # set album
    artist_id = arr_test[3]  # set artist
    
    if user_id != last_user_id:    # if new user reached
        ii = 0                     # reset ii
        user_rating_in_train = numpy.zeros(shape=(6,3))    # reset this array
        
    track_id_vec[ii] = track_id
    album_id_vec[ii] = album_id
    artist_id_vec[ii] = artist_id
    ii = ii + 1
    last_user_id = user_id
    
    if ii == 6:                    # if last entry for current user reached
        while(train_line):
            #for train_line in f_train:
                arr_train = train_line.strip().split('|')    # make list of user_id, item_id, and rating from current line
                train_user_id = arr_train[0]
                train_item_id = arr_train[1]
                train_rating = arr_train[2]
                train_line = f_train.readline()    # go to next line

                if train_user_id < user_id:
                    continue    # return to beginning of while loop
                if train_user_id == user_id:
                    for nn in range(0, 6):    # go through each row in (6,3) user_rating_in_train array for current row in training file
                        if train_item_id == album_id_vec[nn]:    # if album found
                            user_rating_in_train[nn, 0] = train_rating    # set album rating
                        if train_item_id == artist_id_vec[nn]:   # if artist found
                            user_rating_in_train[nn, 1] = train_rating    # set artist rating
                if train_user_id > user_id:   # if we reached the end of the current user, write to output file all the ratings found for current user
                    for nn in range(0, 6):    # go through each row in (6,3) array
                        out_str = str(user_id) + '|' + str(track_id_vec[nn]) + '|' + str(user_rating_in_train[nn,0]) + '|' + str(user_rating_in_train[nn, 1])
                        f_out.write(out_str + '\n')
                    break    # break out of while loop
                    
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Time elapsed - {elapsed_time:.2f} seconds.')

Done! Time elapsed - 24.87 seconds.


In [7]:
# Close all the files
f_test.close()
f_train.close()
f_out.close()

### Part 2 - Create Predicitons file
Here, we use the output file from Part 1 to create a predictions file that will hold the predicitons for each user in the format `userID_trackID,prediction` Prediction will either be 1 or 0, with 1 meaning that we predict the user will like the song and 0 meaning that we predict the user will not like the song. For each user there are 6 predictions (6 songs) and there will be 3 1 predictions and 3 0 predictions.

The way the predictions work is for the 6 songs for each user, we take the album ratings and artist ratings (if found) and apply weights. If apply a higher weight to the album score since it is more likely for a user to like a song in the same album than from the same artist. We then take the top 3 scores and assign these songs as 1 (meaning we think the user will like these songs more) and the other 3 songs are assigned a 0 (meaning we think the user will like these songs less).

In [68]:
# Initialize some values
ratings_array = numpy.zeros(shape=(2,6))
last_user_id = -1
track_id_out_vec = [0] * 6

start_time = time.time()

# Open files
output_file_2 = data_dir + 'Predictions.csv'
f_out_2 = open(output_file_2, 'w')
f_out = open(output_file)

# Write header
f_out_2.write('TrackID,Predictor\n')

# Go through each line of the output file
for line in f_out:
    arr_out = line.strip().split('|')
    user_id_out = arr_out[0]                # set user
    track_id_out = arr_out[1]               # set track
    album_rating = float(arr_out[2]) * .9   # set album and apply weight
    artist_rating = float(arr_out[3]) * .1  # set artist and apply weight
    
    if user_id_out != last_user_id:    # if new user reached
        ii = 0                         # reset ii
        ratings_array = numpy.zeros(shape=(2,6))    # reset this array
        
    if ii < 6:                                 # if still on same user (so from 0 to 5 inclusive)
        ratings_array[0, ii] = album_rating    # add album rating to this vector
        ratings_array[1, ii] = artist_rating   # add artist rating to this vector
        track_id_out_vec[ii] = track_id_out    # add track_id to this vector
        
    ii = ii + 1
    last_user_id = user_id_out
        
    if ii == 6:                                                    # if last entry for current user reached
        ratings_array_sum = numpy.zeros(shape=(6))                 # initialize numpy array for ratings sums
        numpy.sum(ratings_array, axis=0, out=ratings_array_sum)    # sum the album rating and artist rating for each track
        indices = (-ratings_array_sum).argsort()[:3]               # get indices of top 3 scores
        
        # Here we set the predictions by setting a 1 for elements with indices in the indices list and setting a 0 for elements with indices not in the indices list
        predictions = numpy.zeros(shape=(6))    # initlize numpy array for predictions
        for index in indices:
            predictions[index] = 1    
            
        # Here we write to the output file for the 6 track predictions for the current user
        for nn in range(0, 6):
            out_str = str(user_id_out) + '_' + str(track_id_out_vec[nn]) + ',' + str(int(predictions[nn]))
            f_out_2.write(out_str + '\n')

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Time elapsed - {elapsed_time:.2f} seconds.')

f_out_2.close()
f_out.close()

Done! Time elapsed - 0.77 seconds.
